In [ ]:
from quartic_sdk import APIClient, GraphqlClient
from pprint import pprint

true=True
false=False

class Client:

    def __init__(self, url, username, password):
        self._graphql_client = GraphqlClient(url=url,
            username=username, password=password)
        self._api_client = APIClient(host=url,
            username=username, password=password)

    def graphql_client(self):
        return self._graphql_client
    
    def api_client(self):
        return self._api_client
        
    def get_all_entity_types(self):
        """
        """
        gql_query = """
        query MyQuery {
          AhEntityType {
            name
            id
          }
        }
        """
        return self._graphql_client.execute_query(gql_query)["data"]["AhEntityType"]

In [ ]:
#making a Client object; add your username and password in second and third arguement respectively  
client = Client("https://stag.quartic.ai", "dev123", "dev@quartic.ai")

In [ ]:
#queries 

assets_query = '''
    query MyQuery {
        Asset {
            name
            id
        }
    }
'''

sites_query = '''
    query MyQuery {
        Site {
            name
            id
        }
    }
'''

tags_query = '''
    query MyQuery {
        Tag {
            name
            id
        }
    }
'''

entity_query = '''
    query MyQuery {
        AhEntity {
            id
            name
        }
    }
'''
entityTemplate_query = lambda entityType: f'''
    query MyQuery {{
        AhEntityTemplate(entityType: "{entityType}") {{
            id
            name
        }}
    }}
'''

#Mutations

entity_mutation = lambda name, site, entityTemplate, puWcId : f'''
    mutation MyMutation2 {{
    __typename
        AhentityCreate(newAhentity: {{name: "{name}", site: "{site}", entityTemplate: "{entityTemplate}", puWcId: {puWcId}}}) {{
            ok
            ahentity {{
            id
            }}
            errors {{
            messages
            field
            }}
        }}
    }}
'''

asset_mutation = lambda entity_id, tags : f'''
    mutation MyMutation2 {{
    __typename
        AssetCreate(newAsset: {{entity: "{entity_id}", tags: "{tags}"}}) {{
            ok
            asset {{
            id
            }}
            errors {{
            messages
            field
            }}
        }}
    }}
'''

valueTable_query = lambda tag_id : f'''
    query MyQuery2 {{
        __typename
        Tag(id: {tag_id}) {{
            valueTable
        }}
    }}

'''

operation_continuous_mutation = lambda asset_state, stop_val, asset : f'''
    mutation MyMutation {{
        __typename
        AssetOperationCreate(newAssetoperation: {{operationType: A_1, assetStateIdentifier: "{asset_state}", assetStopValue: "{stop_val}", asset: "{asset}"}}) {{
            ok
            errors {{
            messages
            field
            }}
            assetoperationdefinition {{
            id
            }}
        }}
    }}
'''

operation_batch_mutation = lambda batch_phase, batch_id, batch_recipe, start_phase, stop_phase, asset, asset_state, stop_val : f'''
    mutation MyMutation3 {{
        __typename
        AssetOperationCreate(newAssetoperation: {{asset: "{asset}", operationType: A_2, batchTag: "{batch_id}", cyclePhaseTag: "{batch_phase}", cycleTag: "{batch_recipe}", startState: "{start_phase}", stopState: "{stop_phase}", assetStateIdentifier: "{asset_state}", assetStopValue: "{stop_val}"}}) {{
            errors {{
            field
            messages
            }}
            assetoperationdefinition {{
                id
            }}
        }}
    }}
'''

operation_batch_mutation_2 = lambda batch_phase, batch_id,  start_phase, stop_phase, asset, asset_state, stop_val : f'''
    mutation MyMutation3 {{
        __typename
        AssetOperationCreate(newAssetoperation: {{asset: "{asset}", operationType: A_2, batchTag: "{batch_id}", cyclePhaseTag: "{batch_phase}", startState: "{start_phase}", stopState: "{stop_phase}", assetStateIdentifier: "{asset_state}", assetStopValue: "{stop_val}"}}) {{
            errors {{
            field
            messages
            }}
            assetoperationdefinition {{
                id
            }}
        }}
    }}
'''

asset_delete_query = lambda asset_id : f'''
    mutation MyMutation {{
    __typename
        AssetDelete(id: "{asset_id}") {{
            ok
        }}
    }}
'''

operation_delete_query = lambda operation_id : f'''
    mutation MyMutation {{
    __typename
        AssetOperationDelete(id: "{operation_id}") {{
            ok
        }}
    }}
'''

asset_tags_query = lambda asset_id : f'''
    query MyQuery {{
        Asset(id: {asset_id}) {{
            tags {{
                id
            }}
        }}
    }}
'''

asset_entity_query = lambda asset_id : f'''
    query MyQuery {{
        Asset(id: {asset_id}) {{
            entity {{
                id
            }}
        }}
    }}
'''

asset_tags_mutation = lambda asset_id, entity_id, tags : f'''
    mutation MyMutation {{
        __typename
        AssetUpdate(updateAsset: {{id: "{asset_id}", entity: "{entity_id}", tags: "{tags}"}}) {{
            ok
        }}
    }}
'''

In [ ]:
def get_pu_wc():
    return client.graphql_client().execute_query(entity_query)

def get_assetClass():
  
    return client.graphql_client().execute_query(entityTemplate_query(3))

def get_value_table(tag_id):
  
   return client.graphql_client().execute_query(valueTable_query(tag_id))


In [ ]:
@staticmethod
def create_entity(client, name, site, entityTemplate, puWcId):
  
    mutation = entity_mutation(name, site, entityTemplate, puWcId)
    try:
        response =  client.make_request(mutation)
     
        return response['data']['AhentityCreate']['ahentity']['id']
    except Exception as e:
        raise Exception(response) from e

@staticmethod
def parse_tags(client, tag_names):
    tag_list = tag_names.split(',')
    try:
        tag_id_list = [client.tags[tag_name] for tag_name in tag_list]
    except Exception as e:
        raise Exception(f"Failed to parse tags from {tag_names}") from e
    return tag_id_list

@staticmethod
def create_asset(client, entity_id, tags):
    tag_string = ','.join(tags)
    mutation = asset_mutation(entity_id, tag_string)
    response =  client.make_request(mutation)
    try:
        return response['data']['AssetCreate']['asset']['id']
    except Exception as e:
        raise Exception(response) from e
    
@staticmethod
def create_operation(client, **kwargs):
    if kwargs['operation_type'] in [1, "CONTINUOUS_PROCESS"]:
        asset_state_identifier_id = client.tags[kwargs['asset_state_identifier']]
        stop_value_table = client.get_value_table(asset_state_identifier_id)
        stop_value = kwargs['stop_value']

        if stop_value in stop_value_table.keys():
            pass
        elif stop_value in stop_value_table.values():
            stop_value = list(stop_value_table.keys())[list(stop_value_table.values()).index(stop_value)]
        else:
            raise Exception(f"Error: {stop_value} not found in value table for {kwargs['asset_state_identifier']} \n {stop_value_table}")

        mutation = operation_continuous_mutation(asset_state_identifier_id, stop_value, kwargs['asset_id'])
        response = client.make_request(mutation)
        try:
            return response['data']['AssetOperationCreate']['assetoperationdefinition']['id']
        except Exception as e:
            raise Exception(response, ) from e

    elif kwargs['operation_type'] in [2, "BATCH_PROCESS"]:
        asset_state_identifier_id = client.tags[kwargs['asset_state_identifier']]
        stop_value = kwargs['stop_value']
        batch_phase = client.tags[kwargs['batch_phase']]
        batch_id = client.tags[kwargs['batch_id']]
        if not pd.isnull(kwargs['batch_recipe']):
            batch_recipe = client.tags[kwargs['batch_recipe']]
        start_phase = kwargs['start_phase']
        stop_phase = kwargs['stop_phase']
        batch_phase = client.tags[kwargs['batch_phase']]
        batch_phase_value_table = client.get_value_table(batch_phase)
        stop_value_table = client.get_value_table(asset_state_identifier_id)

        if start_phase in batch_phase_value_table.keys():
            pass
        elif start_phase in batch_phase_value_table.values():
            start_phase = list(batch_phase_value_table.keys())[list(batch_phase_value_table.values()).index(start_phase)]
        else:
            raise Exception(f"Error: {start_phase} not found in value table for {kwargs['batch_phase']}")

        if stop_phase in batch_phase_value_table.keys():
            pass
        elif stop_phase in batch_phase_value_table.values():
            stop_phase = list(batch_phase_value_table.keys())[list(batch_phase_value_table.values()).index(stop_phase)]
        else:
            raise Exception(f"Error: {stop_phase} not found in value table for {kwargs['batch_phase']}")

        if stop_value in stop_value_table.keys():
            pass
        elif stop_value in stop_value_table.values():
            stop_value = list(stop_value_table.keys())[list(stop_value_table.values()).index(stop_value)]
        else:
            raise Exception(f"Error: {stop_value} not found in value table for {kwargs['asset_state_identifier']} \n {stop_value_table}")

        mutation = (
            operation_batch_mutation_2(
                batch_phase,
                batch_id,
                start_phase,
                stop_phase,
                kwargs['asset_id'],
                asset_state_identifier_id,
                stop_value,
            )
            if pd.isnull(kwargs['batch_recipe'])
            else operation_batch_mutation(
                batch_phase,
                batch_id,
                batch_recipe,
                start_phase,
                stop_phase,
                kwargs['asset_id'],
                asset_state_identifier_id,
                stop_value,
            )
        )

        response = client.make_request(mutation)
        try:
            logging.info(f"     Done. id: {response['data']['AssetOperationCreate']['assetoperationdefinition']['id']} \n\n")
            return response['data']['AssetOperationCreate']['assetoperationdefinition']['id']
        except Exception as e:
            raise Exception(response) from e

    else:
        raise Exception(f"Error: Operation type {kwargs['operation_type']} is invalid")

@staticmethod
def get_asset_tags(client, asset_name):
    asset_id = client.assets[asset_name]
    query = asset_tags_query(asset_id)
    response = client.make_request(query)
    try:
        tag_id_list = [
            tag['id']
            for tag in response['data']['Asset'][0]['tags']
        ]
        return tag_id_list
    except Exception as e:
        raise Exception(response) from e
            
@staticmethod
def update_asset_tags(client, asset_name, tags):
    asset_id = client.assets[asset_name]        
    tag_string = ','.join(tags)
    entity_id = client.make_request(asset_entity_query(asset_id))['data']['Asset'][0]['entity']['id']
    mutation = asset_tags_mutation(asset_id, entity_id, tag_string)
    response = client.make_request(mutation)
    try:
        return response['data']['AssetUpdate']['ok']
    except Exception as e:
        raise Exception(response) from e
